In [1]:
text = "中兴通讯高级副总裁徐锋：5G时代中兴手机希望重回主流队列, 12月13日，在中兴通讯上海研究所，中兴通讯高级副总裁、中兴终端事业部总裁徐锋在接受澎湃新闻记者采访时表示，5G时代，中兴通讯整体策略是“5G先锋”，希望引领技术创新，包括从系统到终端、完整的5G端到端的解决方案。作为中兴通讯终端业务负责人，徐锋希望5G时代中兴通讯终端能够重新回到主流手机品牌队列。“明年我们在全球会推出10款左右的5G手机，15款以上的5G终端(包括手机和其他形态)。就手机而言，会覆盖2000元到3000元的价位段。”徐锋透露。明年手机市场洗牌是大概率事件徐锋称，2020年是5G手机市场全面爆发的第一年，根据不同的咨询机构预测的情况，加上中兴通讯自己的判断，明年全球5G终端大概会在1.6亿台左右，中国至少占据一半，在8000万到1亿台之间。“所以中国肯定是最主要、最早的市场。5G终端毫无疑问中国市场是主战场。”徐锋表示，中国市场渠道是比较成熟的，中兴通讯线上线下的渠道都会拓展。但他强调，相比4G，运营商渠道在5G初期的主导力会加强，因此跟运营商渠道的合作是非常重要的一环。徐锋认为，明年手机市场洗牌是大概率事件，主要原因是5G技术变化非常大，从4G到5G，技术上是非常大的跨度；另外，应用场景的变化非常大，从纯粹“带宽增加”拓展到“广连接”、“高可靠”、“低时延”的场景，应用场景大大扩展。“在巨大的变化下，也意味着很多机会或者可能出现的问题，所以变局的可能性很大。”那么，中兴手机如何才能突围？徐锋称，手机厂商希望在终端上能够做到领先，最重要的是对5G要有非常深刻的理解。因为5G可能会比以往更多地改变人类的生活、改变社会，所以需要对5G有非常深刻的理解。其次，手机厂商需要利用对5G的理解给用户创造更多的价值，只有抓住这两点才可能领先于别人，比别人更成功。“这是我们希望在5G时代能够突围的方式。当然，传统短板也要弥补，相对来说中兴在国内品牌和渠道方面是相对弱势的，所以这是我们要弥补的。但是靠弥补不可能取胜，只有更多地发挥强项才有可能实现突围。”徐锋1998年加入中兴通讯股份有限公司，2014年加入终端事业部，2018年7月开始担任中兴通讯高级副总裁兼终端事业部总裁，负责中兴全球终端业务。明年推10款5G终端覆盖2000-3000元价位段"

In [1]:
method = "doc_seg_model_spliter"

import sys
sys.path.append("..")
from utils.split_text import *

if method == "cos_sim_spliter":
    from FlagEmbedding import FlagModel
    model = FlagModel('../models/bge-large-zh-v1.5', query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：", use_fp16=True)
    spliter = BaseSpliter.use_subclass("cos_sim_spliter")(model)

elif method == "doc_seg_model_spliter":
    from transformers import AutoModelForTokenClassification, AutoTokenizer
    model_name = '../models/nlp_bert_document-segmentation_chinese-base'
    model = AutoModelForTokenClassification.from_pretrained(model_name, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    spliter = BaseSpliter.use_subclass("doc_seg_model_spliter")(model, tokenizer)

/mnt/disk1/JXH/01_apps/miniforge3/envs/finbot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/disk1/JXH/01_apps/miniforge3/envs/finbot/lib/python3.8/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/mnt/disk1/JXH/01_apps/miniforge3/envs/finbot/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` b

In [2]:
import os
from tqdm import tqdm
def process_stock_news(from_dir, to_dir):
    os.makedirs(to_dir,exist_ok=True)
    done_files = os.listdir(to_dir)
    for filename in os.listdir(from_dir):
        stock_id = filename.split('.')[0]
        if not filename.endswith(".csv") or stock_id+".json" in done_files: continue
        print(f"当前id：{stock_id}")
        filepath = os.path.join(from_dir, filename)
        # 读取CSV文件
        df = pd.read_csv(filepath)
        # 初始化一个空的DataFrame来存储结果
        result = []
        # 处理每行数据
        for index, row in tqdm(df.iterrows()):
                # 假设文件名格式为"000001.csv"
            date = row['Date']
            title = row['Title']
            content = row['Content']
            # 调用spliter.split函数
            chunk_df = spliter.split(content)
            # 将结果合并到原始DataFrame中
            for _, sentence_row in chunk_df.iterrows():
                result.append({
                    'stock_id': stock_id,
                    'date': date,
                    'title': title,
                    'content': content,
                    'chunk': sentence_row['chunk'],
                    'start_idx': sentence_row['start_idx'],
                    'end_idx': sentence_row['end_idx']
                })
        print(f"stock_id {stock_id} chunked.")  # 打印或保存结果DataFrame
        # break
        pd.DataFrame(result).to_json(os.path.join(to_dir,f'{stock_id}.json'), force_ascii=False, orient='records', indent=2)

# 调用函数处理文件夹中的所有文件
process_stock_news('../data/raw/CSI300news','../data/cleaned/CSI300news_chunked')

当前id：600968


8it [00:05,  1.53it/s]


stock_id 600968 chunked.
当前id：600089


7it [00:03,  1.94it/s]


stock_id 600089 chunked.
当前id：002415


...

stock_id 601018 chunked.
当前id：600998


70it [00:48,  1.45it/s]


stock_id 600998 chunked.
当前id：600004


24it [00:11,  2.14it/s]

stock_id 600004 chunked.
